In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time

import glob
import os
import PIL
from tensorflow.keras import layers

In [2]:
tf.__version__

'2.3.0'

In [3]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, input_shape=(100,)))
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(17, activation='tanh'))
    

    return model

In [4]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)


In [5]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(17))

    return model

In [6]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)

tf.Tensor(
[[-0.01261016 -0.01304002 -0.00858982  0.05250838  0.0239376   0.02258762
   0.00818443 -0.01778737  0.01821134  0.01372712  0.04279786  0.01908791
  -0.02929443 -0.02997108  0.00171864 -0.00208553 -0.01173577]], shape=(1, 17), dtype=float32)


In [7]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [8]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [9]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [10]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [11]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [12]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16
BATCH_SIZE = 16
# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [13]:
@tf.function
def train_step(data):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(data, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [14]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for batch in dataset:
      train_step(batch)

    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  #Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

In [15]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  
  

In [16]:
import os 
import earthpy as et 

In [17]:
## Code needed to set a new working directory

#my_path = os.path.join(et.io.HOME, 'recommender_system')
#os.mkdir(my_path)

## Set Working Directory
os.chdir(os.path.join(et.io.HOME, 'recommender_system'))

In [18]:
# Check if it worked
os.getcwd()

'/Users/chriskamerin/recommender_system'

In [19]:
df = pd.read_csv("data_in_use/headphones2hp.csv")

In [20]:
df = df.sample(frac=1).reset_index(drop=True)
df['user_id']  = pd.Categorical(df.user_id).codes
df['headphone_name']  = pd.Categorical(df.headphone_name).codes

In [21]:
cols_to_norm = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance','noise_canceling_importance', 'in_ear_importance',
       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover', 'recording_quality', 'noise_handling','user_rating',
       'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
imp_col = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 
       'price_sensitivity']
#price_col = ['max_price','price']
hp_feat_col = ['recording_quality', 'noise_handling', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
remaining_col = ['Impedance', 'sensitivity(dbv)']

In [22]:
df=df.fillna(0)
df[imp_col] = df[imp_col].apply(lambda x: (x - 0) / (10 - 0))
#df[price_col] = df[price_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df[hp_feat_col] = df[hp_feat_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))   
df[remaining_col] = df[remaining_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))       
#df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [24]:
train(df, EPOCHS)

AttributeError: in user code:

    <ipython-input-13-c03e5b080ea8>:8 train_step  *
        real_output = discriminator(data, training=True)
    /Users/chriskamerin/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:975 __call__  **
        input_spec.assert_input_compatibility(self.input_spec, inputs,
    /Users/chriskamerin/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/input_spec.py:166 assert_input_compatibility
        if x.shape.ndims is None:

    AttributeError: 'str' object has no attribute 'shape'


In [25]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))